In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd
from tensorflow import keras
import tensorflow as tf
import tensorflow_probability as tfp

from matplotlib import font_manager, rc, dates
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

* 메모리

In [2]:
## 불러오기 ##
path = os.getcwd()
site_list = sorted(glob(os.path.join(path,"data","sensing_data","*")))
site_list = list(map(os.path.basename,site_list))
print(path)
print(site_list)
site = "솜사탕"
time_mean = "5T"
csv_by_site_det = []


structure = sorted(glob(os.path.join(path,"data","sensing_data",site,'*')))
structure = list(map(os.path.basename,structure))
for j in structure:
        temp_data = dd.read_csv(os.path.join(path,"data","sensing_data",site,j),encoding='cp949').compute()
        temp_data['T/D'] = pd.to_datetime(temp_data['T/D'])
       
        temp_data.set_index('T/D',inplace=True)
        temp_data = temp_data.resample(time_mean,).mean()
        temp_data['site_details'] = j
        csv_by_site_det.append(temp_data)

data = pd.concat(csv_by_site_det)
data.columns = ['index','temp','hum','pm1','pm2.5','pm4','pm10','co2','tvoc','site_details']
data = data[['temp','hum','pm1','pm2.5','pm4','pm10','co2','tvoc','site_details']]
## 라벨링 ##
# structure
# site_de = "솜사탕_거실.csv"
# data[data.site_details == site_de]
data['vent'] = 0
data['people'] = 0

data.loc[data.between_time('12:00:00', '18:00:00',include_end=False).index,['vent']] = 1
data.loc[data.between_time('9:00:00', '16:00:00',include_end=False).index,['people']] = 1

#0부터 월요일
data.loc[data[data.index.dayofweek == 5].index,['vent']] = 0
data.loc[data[data.index.dayofweek == 5].index,['people']] = 0

data.loc[data[data.index.dayofweek == 6].index,['vent']] = 0
data.loc[data[data.index.dayofweek == 6].index,['people']] = 0

import gym
import torch

from src.Learner.AWAC import AWAC
from src.Learner.DQN import DQN
from src.Learner.Random import DiscreteRandomAgent
from src.nn.MLP import MLP
from src.utils.memory import ReplayMemory
from src.utils.train_utils import prepare_training_inputs

import matplotlib.pyplot as plt 
data_co2 = data.loc[:,['co2','site_details','vent','people']]


## offline dataset 생성 ##
room_name = '솜사탕_거실.csv'

dataset = data_co2[data_co2.site_details == room_name][['co2','vent','people']]

def reward_co2(co2):
    reward = np.exp(-((co2-900)**2)/(2*350**2))
    return reward



# dataset
train_data = dataset.loc['2022-01-02':'2022-01-27']
test_data = dataset.loc['2022-02-13':'2022-02-28']
#
train_data['co2_t+1'] = train_data['co2'].shift(-1)
train_data['people_t+1'] = train_data['people'].shift(-1)
train_data['reward'] = train_data['co2_t+1'].map(lambda x : reward_co2(x))
# train_data = train_data.dropna(axis=0)
train_data['done'] = False
train_data.loc[train_data.between_time('00:00:00', '00:05:00',include_end=False).index,['done']] = True
train_data = train_data.iloc[:-1,:]

test_data['co2_t+1'] = test_data['co2'].shift(-1)
test_data['people_t+1'] = test_data['people'].shift(-1)
test_data['reward'] = test_data['co2_t+1'].map(lambda x : reward_co2(x))
# test_data = test_data.dropna(axis=0)
test_data['done'] = False
test_data.loc[test_data.between_time('00:00:00', '00:05:00',include_end=False).index,['done']] = True
test_data = test_data.iloc[:-1,:]

## 메모리 생성 ##
memory_size = 500000
memory = ReplayMemory(memory_size)
gamma = 0.9
for i, val in enumerate(train_data.values):
    s = np.array([val[0],val[2]])
    a = np.array([val[1]])
    r = np.array([val[5]])
    ns = np.array([val[3],val[4]])
    done = np.array([val[6]])

    experience = (s,ns,a,r,done)
    memory.push(experience)


c:\Users\Go\OneDrive - UOS\allrepos\research_workspace\어린이집데이터_eda
['남부', '동화나라', '소리엘', '솜사탕', '예나']


C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Go\miniconda3\envs\pro1\lib\site-packages\ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

### 예시데이터 모델 제작

* 파라미터

In [189]:
# params of multi_AWAC
agent_num = 5
state_len = 2
hidden_structure= [64, 128, 256, 128, 64]
input_shape= state_len*agent_num
output_shape= 2
act_function= 'relu'
lam = 0.3
optimizer= keras.optimizers.Adam(learning_rate=3 * 1e-4)
gamma= 0.9
tau = 0.1
num_action_samples = 8
loss_fun = tf.keras.losses.MeanSquaredError()

* 기본함수

In [215]:
# dense model 만드는 함수
def make_dense_model(hidden_structure,input_shape,act_function,output_shape):
    inputs = tf.keras.Input(shape=input_shape)
    
    for i, val in enumerate(hidden_structure):
        n_percep = val 
        if i == 0:
            x = keras.layers.Dense(n_percep, activation= act_function)(inputs)

        elif  i != 0 and i != len(hidden_structure)-1:
            x = keras.layers.Dense(n_percep, activation= act_function)(x)

        elif  i == len(hidden_structure)-1:
            x = keras.layers.Dense(n_percep,activation= act_function)(x)
    x = keras.layers.Dense(output_shape)(x)

    dense_model = tf.keras.Model(inputs=inputs, outputs=x)
    return dense_model

# multi_agent structure 만드는 함수
def build_structure(agent_num, hidden_structure,input_shape,act_function,output_shape):
    actor_list = []
    critic_qnet_list = []
    critic_qnet_target_list = []
    for i in range(agent_num):
        actor_list.append(make_dense_model(hidden_structure,input_shape,act_function,output_shape))
        critic_qnet_list.append(make_dense_model(hidden_structure,input_shape,act_function,output_shape))
        critic_qnet_target_list.append(make_dense_model(hidden_structure,input_shape,act_function,output_shape))
    return actor_list, critic_qnet_list, critic_qnet_target_list


def get_action(actor,state, num_samples: int = 3):
    logit_sam = actor(state)
    m = tfp.distributions.Categorical(logits = logit_sam)
    return tf.reshape(m.sample(num_samples),[-1,num_samples])

def get_mean_qsa(qs,sampled_as):
    mean_q = tf.concat([tf.reshape(tf.gather(qs[i],sampled_as[i], axis=0),[1,-1]) for i in range(len(qs))],axis=0)
    mean_q = tf.math.reduce_mean(mean_q,axis=1,keepdims=True)
    return mean_q

* 기본 모델 생성

In [249]:
actor_list, critic_qnet_list, critic_qnet_target_list = build_structure(agent_num, hidden_structure,input_shape,act_function,output_shape)

* 예시 데이터

In [6]:
# 메모리에서 아래와같이 샘플링됐다고 가정하고 테스트(메모리 아직 안만들어짐)
s = np.array([[1,2,1,2,1,2,1,2,1,2,],[2,3,1,2,1,2,3,2,6,7,],[1,4,14,2,1,2,11,2,9,2,]])
ns = np.array([[1,29,1,2,10,2,1,29,1,28,],[17,2,16,2,1,21,1,2,1,22,],[1,21,1,22,1,2,7,2,5,2,]])
a = np.array([[0],[1],[1],])
r = np.array([[2],[4.2],[2.7],])
done = np.array([[0],[0],[0],])
sample_dataset_by_agent = [s, ns, a, r, done]
dataset_list = [sample_dataset_by_agent,sample_dataset_by_agent,sample_dataset_by_agent,sample_dataset_by_agent,sample_dataset_by_agent]


* 개별 critic_qnet agent의 그라디언트 학습

In [217]:
#그래디언트 계산
epoch = 100
for i in range(epoch):
    with tf.GradientTape() as t:
        qs = critic_qnet_target_list[0](ns)
        sampled_as = get_action(actor_list[0],ns, num_action_samples)
        mean_qsa = get_mean_qsa(qs,sampled_as)
        q_target = r + gamma * mean_qsa * (1 - done)

        # 식 3에서 봤던 것처럼 mse형태로 로스함수 설정
        q_val = tf.concat([tf.reshape(tf.gather(critic_qnet_list[0](s)[i],a[i],axis=0),[-1,1]) for i in range(len(s))],axis=0)

        loss = loss_fun(q_val, q_target)
        # print(loss)

        # loss = cal_loss(actor_list[0], critic_qnet_list[0], critic_qnet_target_list[0], s,ns,a,r,done,10,gamma)
        # j_loss = joint_loss(actor_list, critic_qnet_list, critic_qnet_target_list,dataset_list,agent_num)
        # t.watch(j_loss)
        # print([var.name for var in t.watched_variables()])
    grads = t.gradient(loss, critic_qnet_list[0].trainable_variables)
    optimizer.apply_gradients(zip(grads, critic_qnet_list[0].trainable_variables))

* joint loss를 통한 모든 critic_qnet 에이전트 학습

In [218]:
epoch = 100
for i in range(100):
    with tf.GradientTape() as t:
        loss_list =[]
        for j in range(agent_num):
            ##데이터 
            data = dataset_list[j]
            s = data[0]
            ns = data[1]
            a = data[2]
            r = data[3]
            done = data[4]
            ##에이전트
            actor = actor_list[j]
            critic_qnet = critic_qnet_list[j]
            critic_qnet_target = critic_qnet_target_list[j]
            ##로스 계산
            qs = critic_qnet_target(ns)
            sampled_as = get_action(actor,ns, num_action_samples)
            mean_qsa = get_mean_qsa(qs,sampled_as)
            q_target = r + gamma * mean_qsa * (1 - done)

            # 식 3에서 봤던 것처럼 mse형태로 로스함수 설정
            q_val = tf.concat([tf.reshape(tf.gather(critic_qnet(s)[k],a[k],axis=0),[-1,1]) for k in range(len(s))],axis=0)
            loss = loss_fun(q_val, q_target)
            loss_list.append(loss)
        joint_loss = tf.math.reduce_mean(loss_list, axis=None, keepdims=False, name=None)

    critic_qnet_weight_list = [critic_qnet_list[p].trainable_variables for p in range(agent_num)]
    grads = t.gradient(joint_loss, critic_qnet_weight_list)

    for q in range(agent_num):
        optimizer.apply_gradients(zip(grads[q], critic_qnet_weight_list[q]))

* joint loss를 통한 모든 critic_qnet 학습 이후 critic_qnet_target 소프트 업데이트

In [219]:
epoch = 100
for i in range(epoch):
    with tf.GradientTape() as t:
        loss_list =[]
        for j in range(agent_num):
            ##데이터 
            data = dataset_list[j]
            s = data[0]
            ns = data[1]
            a = data[2]
            r = data[3]
            done = data[4]
            ##에이전트
            actor = actor_list[j]
            critic_qnet = critic_qnet_list[j]
            critic_qnet_target = critic_qnet_target_list[j]
            ##로스 계산
            qs = critic_qnet_target(ns)
            sampled_as = get_action(actor,ns, num_action_samples)
            mean_qsa = get_mean_qsa(qs,sampled_as)
            q_target = r + gamma * mean_qsa * (1 - done)

            # 식 3에서 봤던 것처럼 mse형태로 로스함수 설정
            q_val = tf.concat([tf.reshape(tf.gather(critic_qnet(s)[k],a[k],axis=0),[-1,1]) for k in range(len(s))],axis=0)
            loss = loss_fun(q_val, q_target)
            loss_list.append(loss)
        joint_loss = tf.math.reduce_mean(loss_list, axis=None, keepdims=False, name=None)

    critic_qnet_weight_list = [critic_qnet_list[p].trainable_variables for p in range(agent_num)]
    critic_qnet_target_weight_list = [critic_qnet_target_list[p].trainable_variables for p in range(agent_num)]
    grads = t.gradient(joint_loss, critic_qnet_weight_list)

    for q in range(agent_num):
        qnet_weights = critic_qnet_weight_list[q]
        qnet_target_weights = critic_qnet_target_weight_list[q]
        grad = grads[q]

        optimizer.apply_gradients(zip(grad, qnet_weights))

        new_target_weights = []
        for p, target_weights in enumerate(qnet_target_weights):
            qnet_weights_s = qnet_weights[p]
            updated_target_weights_s = target_weights*(1 - tau) + qnet_weights_s*tau
            new_target_weights.append(updated_target_weights_s)
        critic_qnet_target_list[q].set_weights(new_target_weights)



* joint loss로 target까지 업데이트 후 actor 업데이트

In [250]:
epoch = 100
for i in range(epoch):
    with tf.GradientTape() as t:
        loss_list =[]
        for j in range(agent_num):
            ##데이터 
            data = dataset_list[j]
            s = data[0]
            ns = data[1]
            a = data[2]
            r = data[3]
            done = data[4]
            ##에이전트
            actor = actor_list[j]
            critic_qnet = critic_qnet_list[j]
            critic_qnet_target = critic_qnet_target_list[j]
            ##로스 계산
            qs = critic_qnet_target(ns)
            sampled_as = get_action(actor,ns, num_action_samples)
            mean_qsa = get_mean_qsa(qs,sampled_as)
            q_target = r + gamma * mean_qsa * (1 - done)

            # 식 3에서 봤던 것처럼 mse형태로 로스함수 설정
            q_val = tf.concat([tf.reshape(tf.gather(critic_qnet(s)[k],a[k],axis=0),[-1,1]) for k in range(len(s))],axis=0)
            loss = loss_fun(q_val, q_target)
            loss_list.append(loss)
        joint_loss = tf.math.reduce_mean(loss_list, axis=None, keepdims=False, name=None)

    critic_qnet_weight_list = [critic_qnet_list[p].trainable_variables for p in range(agent_num)]
    critic_qnet_target_weight_list = [critic_qnet_target_list[p].trainable_variables for p in range(agent_num)]
    grads = t.gradient(joint_loss, critic_qnet_weight_list)

    for q in range(agent_num):
        # qnet 업데이트
        qnet_weights = critic_qnet_weight_list[q]
        qnet_target_weights = critic_qnet_target_weight_list[q]
        grad = grads[q]

        optimizer.apply_gradients(zip(grad, qnet_weights))
        # target net 업데이트
        new_target_weights = []
        for p, target_weights in enumerate(qnet_target_weights):
            qnet_weights_s = qnet_weights[p]
            updated_target_weights_s = target_weights*(1 - tau) + qnet_weights_s*tau
            new_target_weights.append(updated_target_weights_s)
        critic_qnet_target_list[q].set_weights(new_target_weights)

        #타겟네트워크까지 업데이트한 후 actor net 업데이트
        with tf.GradientTape() as tp:
            # log_probability 계산
            logits = actor_list[q](s)
            m = tfp.distributions.Categorical(logits = logits)
            log_prob = tf.reshape(m.log_prob(a.squeeze()),[-1,1])

            #가중치항 계산
            qs = critic_qnet_target_list[q](s)
            action_probs = tf.nn.softmax(logits, axis=None, name=None)
            vs = tf.math.reduce_sum((qs * action_probs),axis=1, keepdims=True, name=None)
            qas = tf.concat([tf.reshape(tf.gather(qs[k],a[k],axis=0),[-1,1]) for k in range(len(s))],axis=0)
            adv = qas - vs
            weight_term = tf.math.exp((1/lam*adv), name=None)

            #loss
            loss = tf.math.reduce_mean(log_prob * weight_term*-1)
        actor_grad = tp.gradient(loss, actor_list[q].trainable_variables)
        optimizer.apply_gradients(zip(actor_grad, actor_list[q].trainable_variables))
        print(loss)

tf.Tensor(0.45412496, shape=(), dtype=float32)
tf.Tensor(1.0968509, shape=(), dtype=float32)
tf.Tensor(0.58475447, shape=(), dtype=float32)
tf.Tensor(0.7901122, shape=(), dtype=float32)
tf.Tensor(0.5144367, shape=(), dtype=float32)
tf.Tensor(0.5279391, shape=(), dtype=float32)
tf.Tensor(0.56714404, shape=(), dtype=float32)
tf.Tensor(0.60355854, shape=(), dtype=float32)
tf.Tensor(0.46887755, shape=(), dtype=float32)
tf.Tensor(0.36557582, shape=(), dtype=float32)
tf.Tensor(0.40480724, shape=(), dtype=float32)
tf.Tensor(0.39312342, shape=(), dtype=float32)
tf.Tensor(0.5278668, shape=(), dtype=float32)
tf.Tensor(0.40008557, shape=(), dtype=float32)
tf.Tensor(0.2841656, shape=(), dtype=float32)
tf.Tensor(0.2680549, shape=(), dtype=float32)
tf.Tensor(0.32675937, shape=(), dtype=float32)
tf.Tensor(0.41062042, shape=(), dtype=float32)
tf.Tensor(0.37051436, shape=(), dtype=float32)
tf.Tensor(0.22396304, shape=(), dtype=float32)
tf.Tensor(0.22527005, shape=(), dtype=float32)
tf.Tensor(0.2935097,

* 업데이트된 멀티에이전트 모델 액션얻기

In [265]:
# logits = actor_list[0](s)
# m = tfp.distributions.Categorical(logits = logits)
# log_prob = tf.reshape(m.log_prob(a.squeeze()),[-1,1])



#데이터
for i in range(agent_num):
    data = dataset_list[i]
    s = data[0]
    ns = data[1]
    a = data[2]
    r = data[3]
    done = data[4]
    #에이전트
    actor = actor_list[i]
    # 액션
    logits = actor(s)
    actions = tf.math.argmax(logits,axis=1,output_type=tf.dtypes.int64,name=None)
    print(actions)



tf.Tensor([0 1 1], shape=(3,), dtype=int64)
tf.Tensor([0 1 1], shape=(3,), dtype=int64)
tf.Tensor([0 1 1], shape=(3,), dtype=int64)
tf.Tensor([0 1 1], shape=(3,), dtype=int64)
tf.Tensor([0 1 1], shape=(3,), dtype=int64)
